baseline : 직전 월의 데이터로만 했을 때의 결과  
goal : 파산 직전 3개월의 데이터를 사용한 모델  
더 나아가서 파산위험(약 0.7)이 있는 데이터는 balance의 limit를 조정해야할 필요성이 있지않을까  
파산하지 않은 데이터를 통해 limit balance를 예측하는 회귀모델을 만들고  
파산위험이 있는 데이터로 모델을 돌려 나온 결과값과 현재 limit의 차이만큼(결과값이 현재보다 낮은 경우) 조정한다면 [평균 납부금액과 평균 고지금액]

# EDA

### import

In [33]:
import pandas as pd
import numpy as np
import seaborn as sns

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
data = pd.read_csv('/content/drive/MyDrive/AIB/default_clients.csv')

### preprocessing

In [36]:
data

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29997,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29998,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29999,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [37]:
data.columns = data.iloc[0]

In [38]:
data.drop(0,inplace=True)

In [39]:
#id 삭제
data.drop('ID', axis = 1,inplace=True)

In [40]:
#데이터타입 변경
for i in data.columns:
  data[i] = pd.to_numeric(data[i])

In [41]:

#AMT_avg : 6개월 고지금액의 평균
#PAY_avg : 6개월 납부금액의 평균

data['AMT_avg'] = (data['BILL_AMT1']+ data['BILL_AMT2']+ data['BILL_AMT3']) / 3
data['PAY_avg'] = (data['PAY_AMT1']+ data['PAY_AMT2']+ data['PAY_AMT3']) / 3
data['PAY_avg'] = data['PAY_avg'].round(2)
data['AMT_avg'] = data['AMT_avg'].round(2)


In [42]:
#교육수준 1,2,3 이외 모두 통일
def abc(a):
  if a > 3:
    a = 4
  elif a == 0:
    a = 4
  else:
    a = a
  return a

data['EDUCATION'] = data['EDUCATION'].apply(abc)

In [43]:
#결혼 상태 이외값 수정
data['MARRIAGE'] = data['MARRIAGE'].replace(0,3)

In [44]:
#연체 관련 0과 1로 수정
def delay(pay):
  if pay > 0:
    pay = 1
  else:
    pay = 0
  return pay

for i in data.columns[5:11]:
  data[i] = data[i].apply(delay)

In [45]:
#과거 연체 경험 횟수
data['nPay_sum'] = data['PAY_0']+ data['PAY_2']+ data['PAY_3']

In [46]:
dropcol = ['PAY_4','PAY_5','PAY_6',
           'BILL_AMT4','BILL_AMT5','BILL_AMT6',
           'PAY_AMT4','PAY_AMT5','PAY_AMT6']

In [47]:
data.drop(dropcol,axis=1,inplace=True)

In [48]:
def preprocess(df1):
  #컬럼명 변경
  df1.columns = df1.iloc[0]
  df1.drop(0,inplace=True)
  #데이터타입 변경
  for i in df1.columns:
    df1[i] = pd.to_numeric(data[i])

  #최근 3개월 데이터만 사용
  dropcol = ['PAY_4','PAY_5','PAY_6',
           'BILL_AMT4','BILL_AMT5','BILL_AMT6',
           'PAY_AMT4','PAY_AMT5','PAY_AMT6']
  #df1 = df.drop(dropcol,axis=1,inplace=False)
  df1.drop(dropcol,axis=1,inplace=True)

  #AMT_avg : 3개월 고지금액의 평균
  #PAY_avg : 3개월 납부금액의 평균
  df1['AMT_avg'] = (df1['BILL_AMT1']+ df1['BILL_AMT2']+ df1['BILL_AMT3']) / 3
  df1['PAY_avg'] = (df1['PAY_AMT1']+ df1['PAY_AMT2']+ df1['PAY_AMT3']) / 3
  df1['PAY_avg'] = df1['PAY_avg'].round(2)
  df1['AMT_avg'] = df1['AMT_avg'].round(2)

  #교육수준 1,2,3 이외 모두 통일
  def edu(a):
    if a > 3:
      a = 4
    elif a == 0:
      a = 4
    else:
      a = a
    return a

  df1['EDUCATION'] = df1['EDUCATION'].apply(edu)
  
  #결혼 상태 이외값 수정
  df1['MARRIAGE'] = df1['MARRIAGE'].replace(0,3)

  #연체 관련 0과 1로 수정
  def delay(pay):
    if pay > 0:
      pay = 1
    else:
      pay = 0
    return pay

  for i in df1.columns[5:11]:
    df1[i] = df1[i].apply(delay)
  
  #과거 체납 경험 횟수
  df1['nPay_sum'] = df1['PAY_0']+ df1['PAY_2']+ df1['PAY_3']


  return df1

### eda

In [49]:
#중복여부 확인
data.duplicated()

1        False
2        False
3        False
4        False
5        False
         ...  
29996    False
29997    False
29998    False
29999    False
30000    False
Length: 30000, dtype: bool

In [50]:
#결측치 확인
data.isnull().sum()

0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
default payment next month    0
AMT_avg                       0
PAY_avg                       0
nPay_sum                      0
dtype: int64

-----

1. Limit_Bal : 대출 가능 금액  
2. Sex : 성별 (1: 남자, 2: 여자) 
3. Education : 교육수준 (1: 학교졸업, 2: 대학교, 3:고등학교졸업, 4: 그 외)  
4. Marriage : 결혼 여부 (1: 기혼, 2: 미혼, 3: 그 외)
5. 나이  
6. 연체여부 (6개월)  
7. 고지금액 (6개월)  
8. 직전납부금액 (6개월)  
9. 체납 여부 

In [51]:
data.iloc[1,:]

0
LIMIT_BAL                     120000.00
SEX                                2.00
EDUCATION                          2.00
MARRIAGE                           2.00
AGE                               26.00
PAY_0                              0.00
PAY_2                              1.00
PAY_3                              0.00
BILL_AMT1                       2682.00
BILL_AMT2                       1725.00
BILL_AMT3                       2682.00
PAY_AMT1                           0.00
PAY_AMT2                        1000.00
PAY_AMT3                        1000.00
default payment next month         1.00
AMT_avg                         2363.00
PAY_avg                          666.67
nPay_sum                           1.00
Name: 2, dtype: float64

In [52]:
#타겟값 확인
data['default payment next month'].value_counts()

0    23364
1     6636
Name: default payment next month, dtype: int64

In [53]:
data.loc[data['default payment next month'] == 1]['BILL_AMT1'].sum()

321906801

In [54]:
data.loc[data['default payment next month'] == 0]['BILL_AMT1'].sum()

1214793126

In [55]:
data.loc[data['default payment next month'] == 1]['BILL_AMT1'].mean()

48509.162296564195

In [56]:
data.loc[data['default payment next month'] == 0]['BILL_AMT1'].mean()

51994.22727272727

# 모델링

### import

In [57]:
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import eli5
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from eli5.sklearn import PermutationImportance
from sklearn.metrics import roc_auc_score

## Default를 예측하는 분류모델

### sampling

In [59]:
#타겟의 불균형이 우려되 downsampling 진행
data_1 = data.loc[data['default payment next month'] == 1]
data_0 = data.loc[data['default payment next month'] == 0]
# 기존 타겟 0의 수를 랜덤으로 뽑아 비율을 반절로 낮춤
data_0 = data_0.sample(frac = 0.5, replace = False ,random_state = 17)

In [60]:
data_samp = pd.concat([data_0,data_1])

In [61]:
X = data_samp.drop('default payment next month',axis =1)
y = data_samp['default payment next month']

In [62]:
X.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,BILL_AMT1,BILL_AMT2,BILL_AMT3,PAY_AMT1,PAY_AMT2,PAY_AMT3,AMT_avg,PAY_avg,nPay_sum
23590,160000,2,1,1,41,0,0,0,169867,126973,88072,3360,92532,2859,128304.00,32917.00,0
24863,70000,1,2,1,37,0,0,0,68144,69548,70985,2538,2607,2137,69559.00,2427.33,0
26246,130000,2,2,2,30,0,0,0,114838,111226,104301,4224,4415,3600,110121.67,4079.67,0
13326,200000,2,2,1,69,0,0,0,106878,100879,102998,4000,3800,3802,103585.00,3867.33,0
11119,200000,1,2,1,39,0,0,1,162,1879,1707,1879,8,617,1249.33,834.67,1


In [63]:
# 0과 1의 비율이 약 2:1
y.value_counts()

0    11682
1     6636
Name: default payment next month, dtype: int64

### baseline 모델 설정

In [64]:
#체납 직전월의 정보만을 이용한 모델
dropcol_classi_base = ['PAY_2','PAY_3',
           'BILL_AMT2','BILL_AMT3',
           'PAY_AMT2','PAY_AMT3',
           'AMT_avg','PAY_avg','nPay_sum']

X_cl_ba = X.drop(dropcol_classi_base,axis=1)

In [65]:
#기준모델로써 학습모델과 테스트모델로만 나눠 진행
X_ba_train, X_ba_test, y_ba_train, y_ba_test = train_test_split(X_cl_ba,y,test_size = 0.2, random_state=17,stratify = y)

In [66]:
#간단한 결정트리를 사용
model_base = DecisionTreeClassifier(random_state = 17)
model_base.fit(X_ba_train,y_ba_train)

DecisionTreeClassifier(random_state=17)

In [67]:
y_ba_pred = model_base.predict(X_ba_test)

In [68]:
#정확도 점수 확인
accuracy_score(y_ba_test,y_ba_pred)

0.6405567685589519

In [69]:
#리콜점수 확인
print(classification_report(y_ba_test,y_ba_pred))

              precision    recall  f1-score   support

           0       0.72      0.71      0.72      2337
           1       0.50      0.51      0.51      1327

    accuracy                           0.64      3664
   macro avg       0.61      0.61      0.61      3664
weighted avg       0.64      0.64      0.64      3664



---

In [70]:
#최빈값으로만 결정하는 모델의 정확도
baseline_y = np.zeros((len(y),1))
accuracy_score(y,baseline_y)

0.6377333770062233

### 3개월 정보

In [71]:
#분류를 진행할 데이터셋 분리
X_rf_train, X_rf_test, y_rf_train, y_rf_test = train_test_split(X,y,test_size = 0.2, random_state= 17,stratify = y)
X_rf_train, X_rf_val, x_rf_train, x_rf_val = train_test_split(X_rf_train,y_rf_train,test_size = 0.2,random_state = 17, stratify = y_rf_train)

In [72]:
#랜덤포레스트모델 사용
model_rf = RandomForestClassifier(random_state = 17,
                                  class_weight = 'balanced',
                                  n_jobs=-1)

In [73]:
rf_param = {'n_estimators' : range(100,500),
            'max_depth' : range(10,100),
            'min_samples_split' : range(1,100),
            'min_samples_leaf' : range(1,100)
}

clf = RandomizedSearchCV(model_rf,
                         param_distributions = rf_param,
                         scoring='roc_auc',
                         random_state = 17,
                         cv = 5)

clf.fit(X_rf_train,x_rf_train)

RandomizedSearchCV(cv=5,
                   estimator=RandomForestClassifier(class_weight='balanced',
                                                    n_jobs=-1,
                                                    random_state=17),
                   param_distributions={'max_depth': range(10, 100),
                                        'min_samples_leaf': range(1, 100),
                                        'min_samples_split': range(1, 100),
                                        'n_estimators': range(100, 500)},
                   random_state=17, scoring='roc_auc')

In [74]:
clf.best_score_

0.7666064874028067

In [75]:
clf.best_params_

{'max_depth': 84,
 'min_samples_leaf': 39,
 'min_samples_split': 39,
 'n_estimators': 308}

In [76]:
x_rf_pred = clf.predict(X_rf_val)

In [86]:
y_rf_pred = clf.predict(X_rf_test)

In [77]:
clf.predict_proba(X_rf_val)

array([[0.53996153, 0.46003847],
       [0.6563802 , 0.3436198 ],
       [0.23693273, 0.76306727],
       ...,
       [0.55099143, 0.44900857],
       [0.80560819, 0.19439181],
       [0.19504584, 0.80495416]])

In [78]:
print(classification_report(x_rf_val,x_rf_pred))

              precision    recall  f1-score   support

           0       0.80      0.79      0.79      1869
           1       0.64      0.65      0.64      1062

    accuracy                           0.74      2931
   macro avg       0.72      0.72      0.72      2931
weighted avg       0.74      0.74      0.74      2931



In [84]:
roc_auc_score(x_rf_val, x_rf_pred)

0.7209314627901564

In [87]:
roc_auc_score(y_rf_test,y_rf_pred)

0.7021937321661719

In [79]:
accuracy_score(x_rf_val, x_rf_pred)

0.7400204708290685

In [88]:
accuracy_score(y_rf_test,y_rf_pred)

0.7218886462882096

In [80]:
perm_clf = PermutationImportance(clf, random_state=17).fit(X_rf_train,x_rf_train)
eli5.show_weights(perm_clf,feature_names = X_rf_train.columns.tolist() )

Weight,Feature
0.0604 ± 0.0030,nPay_sum
0.0392 ± 0.0012,LIMIT_BAL
0.0320 ± 0.0046,PAY_0
0.0178 ± 0.0012,BILL_AMT1
0.0145 ± 0.0011,PAY_avg
0.0145 ± 0.0012,PAY_AMT3
0.0135 ± 0.0009,BILL_AMT3
0.0124 ± 0.0008,BILL_AMT2
0.0117 ± 0.0011,AMT_avg
0.0113 ± 0.0012,PAY_AMT1


### 함수화

In [81]:
def default(df):
  id = df['ID']
  df1 = df.drop('ID',axis=1,inplace=False)
  pred = clf.predict(df1)
  pred_proba = clf.predict_proba(df1)[:,1]

  dd = pd.DataFrame((id,pred,pred_proba.round(2)),index=('ID','체납 가능 여부','확률')).T

  return dd


In [82]:
def preprocess_1(df1,df2): #df1 : 전처리 후 데이터, df2: default함수 적용 후 데이터
  df1['proba'] = df2['확률']
  data = df1.loc[df1['proba'] > 0.6]
  data1 = data.drop('proba',axis=1,inplace=False)
  data1['ID'] = df2['ID']

  return data


## 회귀모델

### import

In [89]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [90]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

In [91]:
data_lr = data.loc[data['default payment next month'] == 0]

In [92]:
dropcol_lr = ['MARRIAGE','SEX','PAY_0','PAY_2','PAY_3','AMT_avg',
           
           'default payment next month']
data_lr.drop(dropcol_lr,axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [93]:
X_lr = data_lr.drop('LIMIT_BAL',axis =1)
y_lr = data_lr['LIMIT_BAL']

In [94]:
X_lr_train, X_lr_test, y_lr_train, y_lr_test = train_test_split(X_lr,y_lr,test_size=0.2,random_state=17)
X_lr_train, X_lr_val, x_lr_train, x_lr_val = train_test_split(X_lr_train,y_lr_train,test_size=0.2, random_state= 17)

### baseline

In [95]:
y_ba_pred = np.zeros((len(y_lr),1))

In [96]:
y_ba_pred[:] = y_lr.mean()

In [98]:
y_lr.mean()

178099.72607430234

In [97]:
mean_absolute_error(y_lr,y_ba_pred)

106401.3581416409

### Regression

In [99]:
pipe = make_pipeline(StandardScaler(),
                     LinearRegression())

In [100]:
pipe.fit(X_lr_train,x_lr_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [101]:
pipe.score(X_lr_val,x_lr_val)

0.21628251477837168

In [102]:
pipe.get_params()

{'linearregression': LinearRegression(),
 'linearregression__copy_X': True,
 'linearregression__fit_intercept': True,
 'linearregression__n_jobs': None,
 'linearregression__normalize': 'deprecated',
 'linearregression__positive': False,
 'memory': None,
 'standardscaler': StandardScaler(),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'steps': [('standardscaler', StandardScaler()),
  ('linearregression', LinearRegression())],
 'verbose': False}

In [103]:
pipe_xgb = make_pipeline(#StandardScaler(),
                         xgb.XGBRegressor(objective='reg:squarederror'))

In [104]:
pipe_xgb.fit(X_lr_train,x_lr_train)

Pipeline(steps=[('xgbregressor', XGBRegressor(objective='reg:squarederror'))])

In [105]:
pipe_xgb.score(X_lr_val,x_lr_val)

0.4006170241728203

In [106]:
xgb_params = {'xgbregressor__n_estimators' : range(10,1000),
              'xgbregressor__max_depth' : range (3,30),
              'xgbregressor__eta' : [0.3,0.1,0.01]}

In [107]:
pipe_xgb.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('xgbregressor', XGBRegressor(objective='reg:squarederror'))])>

In [108]:
xgb = RandomizedSearchCV(pipe_xgb,
                         param_distributions = xgb_params,
                         n_iter=10,
                         scoring='r2',
                         cv=5,
                         random_state=17)

In [109]:
xgb.fit(X_lr_train,x_lr_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('xgbregressor',
                                              XGBRegressor(objective='reg:squarederror'))]),
                   param_distributions={'xgbregressor__eta': [0.3, 0.1, 0.01],
                                        'xgbregressor__max_depth': range(3, 30),
                                        'xgbregressor__n_estimators': range(10, 1000)},
                   random_state=17, scoring='r2')

In [110]:
xgb.best_score_

0.40970833795523365

In [111]:
xgb.best_params_

{'xgbregressor__eta': 0.01,
 'xgbregressor__max_depth': 4,
 'xgbregressor__n_estimators': 528}

In [112]:
x_val_pred = xgb.predict(X_lr_val)

In [114]:
y_pred = xgb.predict(X_lr_test)

In [113]:
r2_score(x_lr_val,x_val_pred)

0.39697836188034763

In [115]:
r2_score(y_lr_test,y_pred)

0.3905874918009604

In [116]:
mean_absolute_error(y_lr_test,y_pred)

75722.6633498006

In [117]:
perm_rg = PermutationImportance(xgb, random_state=17).fit(X_lr_train,x_lr_train)
eli5.show_weights(perm_rg,feature_names = X_lr_train.columns.tolist() )

Weight,Feature
0.1874 ± 0.0046,BILL_AMT1
0.1627 ± 0.0090,AGE
0.1431 ± 0.0068,PAY_avg
0.1041 ± 0.0045,PAY_AMT1
0.1020 ± 0.0040,BILL_AMT2
0.0941 ± 0.0017,BILL_AMT3
0.0716 ± 0.0008,PAY_AMT3
0.0668 ± 0.0050,EDUCATION
0.0603 ± 0.0031,PAY_AMT2
0.0145 ± 0.0010,nPay_sum


### 함수화

In [119]:
def balance(df):
  id = df['ID'].tolist()
  dropcol_lr = ['MARRIAGE','SEX','PAY_0','PAY_2','PAY_3','AMT_avg','proba','ID']
           
           
  
  df1 = df.drop(dropcol_lr,axis=1,inplace=False)
  X_lr = df1.drop('LIMIT_BAL',axis =1)
  y_lr = df1['LIMIT_BAL']

  y_pred = xgb.predict(X_lr).round(0)

  prev_bal = df1['LIMIT_BAL'].tolist()
  adjust = pd.DataFrame((prev_bal,y_pred.round(0)),index=('조정 전 한도','조정 후 한도')).T
  adjust['조정 금액'] = adjust['조정 전 한도'] - adjust['조정 후 한도']
  adjust['ID'] = id
  adjust = adjust[['ID','조정 전 한도','조정 후 한도','조정 금액']]

  def zero(n):
    if n < 0:
      n = 0
  
    return n
  
  adjust['조정 금액'] = adjust['조정 금액'].apply(zero)

  return adjust

## 활용

In [120]:
data = pd.read_csv('/content/drive/MyDrive/AIB/default_clients.csv')

In [121]:
#전처리
rrr= preprocess(data)

In [122]:
#타겟값이 없는 데이터를 가정
rrr.drop('default payment next month', axis = 1 ,inplace= True)

In [123]:
rrr

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,BILL_AMT1,BILL_AMT2,BILL_AMT3,PAY_AMT1,PAY_AMT2,PAY_AMT3,AMT_avg,PAY_avg,nPay_sum
1,1,20000,2,2,1,1,1,1,0,1,1,689,0,689,0,2568.00,229.67,2
2,2,120000,2,2,2,1,0,1,0,1,1,2682,0,1000,1000,2363.00,666.67,1
3,3,90000,2,2,2,1,0,0,0,1,1,13559,1518,1500,1000,18941.67,1339.33,0
4,4,50000,2,2,1,1,0,0,0,1,1,49291,2000,2019,1200,48171.33,1739.67,0
5,5,50000,1,2,1,1,0,0,0,1,1,35835,2000,36681,10000,16707.33,16227.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,29996,220000,1,3,1,1,0,0,0,1,1,208365,8500,20000,5003,196709.33,11167.67,0
29997,29997,150000,1,3,2,1,0,0,0,1,1,3502,1837,3526,8998,2337.67,4787.00,0
29998,29998,30000,1,2,2,1,1,1,1,1,1,2758,0,0,22000,3226.33,7333.33,3
29999,29999,80000,1,3,1,1,1,0,0,0,1,76304,85900,3409,1178,51012.67,30162.33,1


In [124]:
#체납 확률과 여부
ggg = default(rrr)

In [129]:
ggg

,ID,체납 가능 여부,확률
1,1.0,1.0,0.60
2,2.0,0.0,0.40
3,3.0,0.0,0.42
4,4.0,0.0,0.34
5,5.0,0.0,0.41
...,...,...,...
29996,29996.0,0.0,0.37
29997,29997.0,1.0,0.72
29998,29998.0,1.0,0.54
29999,29999.0,0.0,0.43


In [126]:
#확률에 근거한 한도조정이 필요한 ID 탐색
kkk = preprocess_1(rrr,ggg)
kkk

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,BILL_AMT1,BILL_AMT2,BILL_AMT3,PAY_AMT1,PAY_AMT2,PAY_AMT3,AMT_avg,PAY_avg,nPay_sum,proba
13,13,630000,2,2,2,1,0,0,0,1,1,6500,1000,6500,6500,8379.00,4666.67,0,0.77
15,15,250000,1,1,2,1,0,0,0,1,1,63561,3000,3000,3000,67169.33,3000.00,0,0.65
16,16,50000,2,3,3,1,1,1,0,1,1,28116,0,1500,1100,35967.67,866.67,2,0.62
18,18,320000,1,1,1,1,0,0,0,1,1,194663,10358,10000,75940,231495.00,32099.33,0,0.64
22,22,120000,2,2,1,1,0,0,0,1,1,316,316,316,0,316.00,210.67,0,0.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29976,29976,220000,1,2,1,1,0,0,0,1,1,1369,8840,6643,5924,18428.00,7135.67,0,0.77
29981,29981,50000,1,3,2,1,0,0,0,1,1,49397,10000,4000,5000,47506.00,6333.33,0,0.80
29991,29991,140000,1,2,1,1,0,0,0,1,1,139110,6000,7000,4228,138192.33,5742.67,0,0.82
29994,29994,100000,1,1,2,1,0,0,0,1,1,102996,2000,111784,4000,35821.67,39261.33,0,0.77


In [127]:
#한도 조정
zzz = balance(kkk)

In [128]:
zzz

,ID,조정 전 한도,조정 후 한도,조정 금액
0,13,630000.0,142996.0,487004.0
1,15,250000.0,89614.0,160386.0
2,16,50000.0,36621.0,13379.0
3,18,320000.0,132688.0,187312.0
4,22,120000.0,81017.0,38983.0
...,...,...,...,...
5363,29976,220000.0,59979.0,160021.0
5364,29981,50000.0,3914.0,46086.0
5365,29991,140000.0,42776.0,97224.0
5366,29994,100000.0,20370.0,79630.0
